In [2]:
!pip install scikit-learn matplotlib seaborn

   ---------------------------------------- 0.0/11.0 MB ? eta -:--:--
   --- ------------------------------------ 1.0/11.0 MB 5.6 MB/s eta 0:00:02
   ------- -------------------------------- 2.1/11.0 MB 4.7 MB/s eta 0:00:02
   ----------- ---------------------------- 3.1/11.0 MB 4.7 MB/s eta 0:00:02
   --------------- ------------------------ 4.2/11.0 MB 4.8 MB/s eta 0:00:02
   -------------------- ------------------- 5.5/11.0 MB 5.0 MB/s eta 0:00:02
   ----------------------- ---------------- 6.6/11.0 MB 5.0 MB/s eta 0:00:01
   ---------------------------- ----------- 7.9/11.0 MB 5.1 MB/s eta 0:00:01
   -------------------------------- ------- 8.9/11.0 MB 5.2 MB/s eta 0:00:01
   ------------------------------------ --- 10.0/11.0 MB 5.2 MB/s eta 0:00:01
   ---------------------------------------  10.7/11.0 MB 5.2 MB/s eta 0:00:01
   ---------------------------------------- 11.0/11.0 MB 4.7 MB/s eta 0:00:00
   ---------------------------------------- 0.0/7.8 MB ? eta -:--:--
   ------ -

In [12]:
!pip install openpyxl

In [3]:
import pandas as pd
from sklearn.cluster import KMeans
from sklearn.preprocessing import StandardScaler
import matplotlib.pyplot as plt
import seaborn as sns

Matplotlib is building the font cache; this may take a moment.


In [13]:
ventas = pd.read_excel('C:/Users/james/OneDrive/Escritorio/TareaDiscreta/MD_tarea_2/ventasOrdenado.xlsx')  # Datos de ventas
kits = pd.read_excel('C:/Users/james/OneDrive/Escritorio/TareaDiscreta/MD_tarea_2/kits.xlsx')   # Datos de kits

In [17]:
def desagregar_ventas(ventas, kits):
    # Unir las ventas con los kits según el código de artículo
    ventas_desagregadas = ventas.copy()
    
    # Desagregar kits por sus componentes
    kits_limpio = kits.melt(id_vars='Código KIT (Sin historial)', 
                            value_vars=['Código 1', 'Código 2', 'Código 3'],
                            var_name='Componente', value_name='articulo')
    kits_limpio.dropna(inplace=True)

    # Unir la tabla de ventas con la tabla de kits
    ventas_con_kits = pd.merge(ventas_desagregadas, kits_limpio, 
                               left_on='articulo', right_on='articulo', how='left')
    
    # Ajustar las ventas del kit a cada artículo componente
    ventas_con_kits['Ventas Ajustadas'] = ventas_con_kits.apply(
        lambda x: x['venta_usd'] if pd.isna(x['Código KIT (Sin historial)']) else x['venta_usd'] / 3, axis=1
    )
    
    return ventas_con_kits[['articulo', 'Ventas Ajustadas', 'Código KIT (Sin historial)']]



In [21]:
import pandas as pd

# Load the uploaded Excel file to inspect its content
file_path = 'C:/Users/james/OneDrive/Escritorio/TareaDiscreta/MD_tarea_2/ventasOrdenado.xlsx'

# Load the Excel file to check available sheets and structure
ventas_data = pd.ExcelFile(file_path)
ventas_data.sheet_names

['Sheet1']

In [26]:
ventas_df = pd.read_excel(file_path, sheet_name='Sheet1', nrows=18,)
ventas_df.head()

,cia,fecha,codigo,razon_social,rubro,departamento,documento,articulo,nombre_articulo,fuente_suministro,cantidad,venta_usd,venta_pen,costo,vendedor,cobrador,marca,transacciones
0,ic,2021-01-04,c08724,casas sovero daniela yamilet,"ventas de partes, piezas y accesorios para veh...",lima,f001-01-0034897,48320.0,focos h1 12v 55w,narva,50,51.27,185.80,134.87000,bayona paiva myriam,vega zúñiga enrique,no registro,1
1,ic,2021-01-04,c08724,casas sovero daniela yamilet,"ventas de partes, piezas y accesorios para veh...",lima,f001-01-0034897,48901.0,focos h4 12v 100x90 rallye(7),narva,300,597.46,2165.20,1413.05010,bayona paiva myriam,vega zúñiga enrique,no registro,1
2,ic,2021-01-04,c10722,gc service & parts s.a.c.,NaN,lima,f001-01-0034900,NaN,filtro de aire 1° af25729 055134r1,mann - brasil,8,152.54,552.80,310.15000,rojas vasquez lilian,oficina,no registro,1
3,ic,2021-01-05,c12849,lopez rosales yolanda juana,NaN,junin,f001-01-0034918,NaN,pastillas de freno ceramica d1567 delt,wagner - pastillas,2,42.36,153.68,96.56000,cañas flores alan joel,cañas flores alan joel,no registro,1
4,ic,2021-01-06,c11327,palante flotas s.a.c.,NaN,lima,f001-01-0034930,NaN,aceite sae20w50 sn racing,valvoline,90,861.86,3129.41,2323.67004,mostrador 4,olórtegui manrique juan,no registro,1


In [31]:
ventas_fd = ventas_fd.drop(['razon_social', 'documento', 'marca', 'transacciones'], axis=1)
print(ventas_fd)

NameError: name 'ventas_f' is not defined

In [18]:
ventas_desagregadas = desagregar_ventas(ventas, kits)
print("Ventas desagregadas:")
print(ventas_desagregadas.head())

KeyError: 'Artículo'

In [ ]:
def preparar_datos_para_clustering(ventas_desagregadas):
    # Crear una tabla con las ventas agregadas por artículo
    ventas_por_articulo = ventas_desagregadas.groupby('Código Artículo').sum().reset_index()

    # Escalar los datos para el clustering
    scaler = StandardScaler()
    ventas_scaled = scaler.fit_transform(ventas_por_articulo[['Ventas Ajustadas']])
    
    return ventas_por_articulo, ventas_scaled

In [ ]:
def aplicar_clustering(ventas_scaled, n_clusters=5):
    kmeans = KMeans(n_clusters=n_clusters, random_state=42)
    clusters = kmeans.fit_predict(ventas_scaled)
    return clusters

In [ ]:
ventas_por_articulo, ventas_scaled = preparar_datos_para_clustering(ventas_desagregadas)
clusters = aplicar_clustering(ventas_scaled)

In [ ]:
ventas_por_articulo['Cluster'] = clusters
print("\nVentas por artículo con clusters:")
print(ventas_por_articulo.head())

In [ ]:
plt.figure(figsize=(10, 6))
sns.scatterplot(x='Código Artículo', y='Ventas Ajustadas', hue='Cluster', 
                data=ventas_por_articulo, palette='Set1', legend='full')
plt.title('Clusters de artículos según ventas ajustadas')
plt.xticks(rotation=90)
plt.show()